In [11]:
import argparse

from fscve_step1_db import FscveStep1DbHelper
from pyutilszxy import makedirs
import pandas as pd
from pyutilszxy import stringtomd5
import linecache
from fscve_step1_db import CrashAnalysis
import sys
import subprocess
import re
import os
import time
import glob
import shlex
import shutil
import threading
from time import sleep
from pyutilszxy import walk_files

dbhelper = FscveStep1DbHelper( "mp42aac", "fcb", 2, False)
session = dbhelper.session
expid = dbhelper.experiment_id
benchmark = dbhelper.benchmark
fc = dbhelper.fc
# query = session.query(AnalysisState.analysis_id,AnalysisState.discovery_id).first()
# print(query)
print(f"{benchmark}---{fc}-----{expid}")
shutil.rmtree("tmp", ignore_errors=True)
makedirs("tmp", mode=0o777, ignore_errors=False, exist_ok=True)
shutil.rmtree("crashes_analysis", ignore_errors=True)
makedirs("crashes_analysis", mode=0o777, ignore_errors=False, exist_ok=True)

mp42aac---fcb-----1


In [12]:
from pyutilszxy import walk_files
import pandas as pd
# total number of unique crashe testcases,maybe the original crashes would lead to the same crashes ,the number is still essential for evaluating fuzzer combinations ,because some testcase would not be correctly validated by the addresssanitizer and other tools
wav_path = f"runinfosqlites/{benchmark}/{fc}/{expid}/crashes"
FILES = walk_files(wav_path)
if not FILES:
    print("[Error] empty crash files? please check glob syntax!")
    exit(1)
serie_unique_crashes=pd.Series([benchmark, fc, expid, len(FILES)],index=['benchmark','fuzzer_combination','experiment_id','count'])
print(serie_unique_crashes)
df_unique_crashes = pd.DataFrame()

# transform serie to dataframe
stmp = serie_unique_crashes.to_frame()
s2 = pd.DataFrame(stmp.values.T, columns=stmp.index)
df_unique_crashes = pd.concat([df_unique_crashes,s2], ignore_index=True)
print(df_unique_crashes)
path_unique_crashes_csv = f"csv/fscve_unique_crashes_{benchmark}_{fc}_{expid}.csv"
df_unique_crashes.to_csv(path_unique_crashes_csv)

benchmark             mp42aac
fuzzer_combination        fcb
experiment_id               1
count                     151
dtype: object
  benchmark fuzzer_combination experiment_id count
0   mp42aac                fcb             1   151


In [13]:
BIN2ARGS = {
    "addr2line": "-e @@",
    "ar": "-t @@",
    "strings": "-d @@",
    "size": "@@",
    "c++filt": "@@",
    # "nm-new": "-a -C -l --synthetic @@",
    # "nm": "-a -C -l --synthetic @@",
    # "nm": "-a -C -l --synthetic @@",
    "nm": " -C @@",
    # "objdump": "--dwarf-check -C -g -f -dwarf -x @@",
    "objdump": "-d @@",
    # "readelf": "-a -c -w -I @@",
    "readelf": "-a @@",
    # "strip-new": "-o /dev/null -s @@",
    "strip": "-o /dev/null -s @@",
    "xml": "@@",
    "gnuplot": "@@",
    "boringssl": "@@",
    "c-ares": "@@",
    "freetype2": "@@",
    "guetzli": "@@",
    "harfbuzz": "@@",
    "json": "@@",
    "lcms": "@@",
    "libarchive": "@@",
    "libjpeg-turbo": "@@",
    "libpng": "@@",
    "libssh": "@@",
    "libxml2": "@@",
    "llvm-libcxxabi": "@@",
    "openssl-1.0.1f": "@@",
    "openssl-1.0.2d": "@@",
    "openssl-1.1.0c": "@@",
    "openssl-1.1.0c-bignum": "@@",
    "openssl-1.1.0c-x509": "@@",
    "openthread": "@@",
    "openthread-ip6": "@@",
    "openthread-radio": "@@",
    "pcre2": "@@",
    "proj4": "@@",
    "re2": "@@",
    "sqlite": "@@",
    "vorbis": "@@",
    "woff2": "@@",
    "wpantund": "@@",
    "base64": "-d @@",
    "md5sum": "-c @@",
    "uniq": "@@",
    "who": "@@",
    # zhaoxy add for unifuzz dataset start
    #zhaoxy add for unifuzz dataset start
    "cflow": "@@",
    "flvmeta": "@@",
    "infotocap": "-o /dev/null @@",
    "jhead": "@@",
    "jq": ". @@",
    "exiv2": "@@",
    "mp42aac": "@@ /dev/null",
    "wav2swf": "-o /dev/null @@",
    #"mp3gain": "@@"
    #zhaoxy add for unifuzz dataset end

    #zhaoxy add for putone mp3gain
    "mp3gain": "@@",
    # zhaoxy add for putonefile_magic
    "file_magic": "2147483647",
    "freetype2-2017":  "@@",
    "file_magic_fuzzer":  "@@",
    #zhaoxy add for magma
    "libpng": "@@",
    "libtiff": "@@",
    "libsndfile": "@@",
    "libxml2": "@@",
    "lua": "@@",
    "openssl": "@@",
    "as": "@@",
    "ld": "@@",
    "ranlib": "@@",
    "gprof": "@@",
}

# print(f"{BIN2ARGS['nm']}")
# para = BIN2ARGS[benchmark]
# cmd = f"./benchmarks/{benchmark} {para}"

# print(cmd)

In [14]:
"""
Assume that we have conducted experiments with 30 repetitions and the folder is like:
objdump/fca/1/crashes
We can run the crash to obtain ASAN output to folder objdump/fca/1/crashes
crashrunner.py
"""

MAX_THREADS = 10
os.environ[
    "ASAN_OPTIONS"] = 'stack_trace_format="FUNCTIONSTART%fFUNCTIONEND----LOCATIONSTART%SLOCATIONEND----FRAMESTART%nFRAMEEND"'


# use the following environment variable
# os.environ["ASAN_SYMBOLIZER_PATH"] = 'ASAN_SYMBOLIZER_PATH= /usr/bin/llvm-symbolizer'

# hign speed,recommended
# delete all dirs and file under directory dirpath
def delete_dirs_and_files(dirpath):
    for root, dirs, files in os.walk(dirpath, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
        os.rmdir(root)


def dprint(*args):
    sys.stderr.write(" ".join([str(i) for i in args]) + "\n")


def run_one_file(file, cmd, tmpfile, stdoutfile, stderrfile, timeoutfile, timeout=10):
    """
    Run certain file to get stdoutfile and stderrfile
    First, the file will be copied to tmpfile,
    then @@ in cmd will be replaced to tmpfile,
    output will be saved to stdoutfile and stderrfile
    if timedout, timeoutfile will be created

    Return: (nottimeout, runtime, outputtext)

    The caller should keep tmpfile only operated by current thread,
    stdoutfile folder should be present
    """
    shutil.copy(file, tmpfile)

    if "@@" in cmd:
        cmds = shlex.split(cmd.replace("@@", tmpfile))
        stdin = None
    else:
        cmds = shlex.split(cmd)
        stdin = open(tmpfile, "rb")

    nottimeout = True
    if os.path.exists(timeoutfile):
        os.unlink(timeoutfile)
    starttime = time.time()

    # dprint(cmds)
    try:
        x = subprocess.run(cmds, stdin=stdin, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=timeout)
        exitcode = x.returncode
    except subprocess.TimeoutExpired as e:
        x = e
        nottimeout = False
        with open(timeoutfile, "w") as tmp:
            tmp.write(file + "\n")
        exitcode = -15  # SIGTERM

    endtime = time.time()
    runtime = endtime - starttime
    if x.stdout is None or x.stderr is None:
        return 1, 0, 0, "empty outputtext"

    outputtext = x.stdout.decode(errors="ignore") + "\n" + x.stderr.decode(errors="ignore")

    with open(stdoutfile, "wb") as fp:
        fp.write(x.stdout)
    with open(stderrfile, "wb") as fp:
        fp.write(x.stderr)
    with open(stdoutfile.replace(".stdout", ".returncode"), "w") as fp:
        fp.write(str(exitcode))

    return nottimeout, exitcode, runtime, outputtext


FINISHED = 0
RESULT = {}
output_folder = ""


def thread_main(files, cmd, threadid, myname):
    # in each thread, iteratively call run_one_file:
    #     run_one_file(file, cmd, tmpfile, stdoutfile, stderrfile, timeoutfile, timeout=10)
    # tmpfile is calculated using myname and threadid
    # pathname of other output files are generated using file pathname,
    # appending ".stdout", ".stderr", ".timeout" suffix respectively

    global FINISHED, RESULT, output_folder
    pwd = os.getcwd()

    for file in files:
        # we will place output files to a folder under crash_ananlysis
        # this folder is generated solely from file pathname
        # used as a cache folder, to speed up further analysis
        # we ignore certain keywords to shorten output_folder name

        # # print(file)
        fnames = file.split("/")
        fname = fnames[-1]

        if not os.path.exists(output_folder):
            os.makedirs(output_folder, exist_ok=True)

        tmpfile = "{myname}_{threadid}".format(**locals())
        stdoutfile = output_folder + fname + ".stdout"
        stderrfile = output_folder + fname + ".stderr"
        timeoutfile = output_folder + fname + ".timeout"

        # res: (nottimeout, exitcode, runtime, outputtext)
        # do not read cache, run it!
        res = run_one_file(file, cmd, tmpfile, stdoutfile, stderrfile, timeoutfile, timeout=10)

        # print("zxy crash cmd result", res)
        RESULT[file] = res
        # if "AddressSanitizer" in res[3]:
        #     print(file)
        FINISHED += 1


def exec_crash_testcase_and_save_result():
    global output_folder
    wav_path = f"runinfosqlites/{benchmark}/{fc}/{expid}/crashes"
    FILES = walk_files(wav_path)
    if not FILES:
        print("[Error] empty crash files? please check glob syntax!")
        exit(1)

    len_FILES = len(FILES)
    dprint("Total files:", len_FILES)

    # cmd = os.environ.get("CMD", None)
    # if not cmd:
    #     print("[Error] env CMD not given")
    #     exit(1)
    # cmd = f"./benchmarks/{benchmark} {BIN2ARGS[{benchmark}]}"
    para = BIN2ARGS[benchmark]
    cmd = f"./benchmarks/asan/{benchmark} {para}"
    progpath = shlex.split(cmd)[0]
    print(progpath)
    progname = progpath.split("/")[-1]
    print(progname)

    assert os.access(progpath, os.X_OK), "CMD program not executable?"

    myname = "tmp/tmp_crashrunner_" + str(os.getpid())

    threadN = min(MAX_THREADS, len_FILES)

    output_folder = os.getcwd() + "/crashes_analysis/"

    if os.path.exists(output_folder):
        delete_dirs_and_files(output_folder)

    for i in range(threadN):
        t = threading.Thread(target=thread_main, args=[FILES[i::threadN], cmd, i, myname])
        t.start()

    while FINISHED < len_FILES:
        print("finished:", FINISHED, "/", len_FILES)
        sleep(2)

    foundbugids = set()
    for name, value in RESULT.items():
        text = value[3]
        print(name, "*" * 30, text)
        # if "AddressSanitizer" in text:
        #     foundbugids.add(getbugid(text, progname))
    print("bugids:", sorted(list(foundbugids)))

    for f in glob.glob(myname + "*"):
        os.unlink(f)


In [15]:
if __name__ == "__main__":
    exec_crash_testcase_and_save_result()

Total files: 151


./benchmarks/asan/mp42aac
mp42aac
finished: 0 / 151
finished: 0 / 151
finished: 0 / 151
finished: 0 / 151
finished: 0 / 151
finished: 0 / 151
finished: 0 / 151
finished: 94 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 100 / 151
finished: 138 / 151
runinfosqlites/mp42aac/fcb/1/crashes/bb2f0449d9c05a3517e9919364044a8ed9f3848497c5947dc7c0c7d52dea178a ****************************** empty outputtext
runinfosqlites/mp42aac/fcb/1/crashes/d59ff9de8300fd9af296099b5b360fe267a6559e7e0f301fea2264bf8479ec1f ****************************** empty outputtext
runinfosqlites/mp42aac/fcb/1/crashes/5f3e5ebc5d4c1dc136dc20171fd9378ce3477be2df2f571a4ac5d011b317231c ****************************** empty ou

In [16]:
# #os.environ["ASAN_SYMBOLIZER_PATH"] = 'ASAN_SYMBOLIZER_PATH=/usr/bin/llvm-symbolizer-9'
# res = run_one_file("/home/kakaxdu/prjdataspell/casefc/runinfosqlites/nm/fca/1/crashes/0a1a3fd715470e3fdf552533e1758b8c88e450b8c05211b43a733b751ab3ebb4", "./benchmarks/aflasan/nm -C @@", "tmp/tmp_crashrunner_1239690_0", "test.out", "test.err", "test.timeout", timeout=10)
# print(f"zxy crash cmd result nottimeout:{res[0]} exitcode:{res[1]}" ,res)

In [17]:

# fetch all crash files and save the vulerability to database table:crashs
def analysis_crash_exec_result():
    global RESULT

    # drop table CrashAnalysis to store new data
    session.query(CrashAnalysis).delete()
    session.commit()

    wav_path = f"runinfosqlites/{benchmark}/{fc}/{expid}/crashes"
    analysis_path = os.getcwd() + "/crashes_analysis/"
    FILES = walk_files(wav_path)
    print("111111111111")
    if not FILES:
        print("[Error] empty crash files? please check glob syntax!")
        exit(1)
    test_case_hashes = [f.split("/")[-1] for f in FILES]
    print("22222222")
    for f in FILES:
        if RESULT[f][1]!=0 and RESULT[f][0]!=0 : #real bug #[1]exitcode,[0]nottimeout
            print("333333333333333333")
            try:
                lines = linecache.getlines("{}{}.stderr".format(analysis_path,f.split("/")[-1]))
                print(lines)
                frame_start = 0
                type_index = 0
                frame_end = 0
                for i in range(0,len(lines)):
                    if lines[i].find("ERROR: AddressSanitizer")!=-1:   #fetch top 3 frames to decide the unique vulnerability
                        type_index = i
                    if lines[i].find("FRAMESTART0FRAMEEND")!=-1:   #fetch top 3 frames to decide the unique vulnerability
                        frame_start = i
                    if lines[i].find("FRAMESTART3FRAMEEND")!=-1:   #fetch top 3 frames to decide the unique vulnerability
                        frame_end = i
                        break
                bug_line = lines[type_index]
                bug_type = re.findall(r"ERROR: AddressSanitizer: ([-a-zA-Z]+)\b", bug_line)  # eg.stack-overflow
                if len(bug_type)==0: # for bug_type[0] out of bound
                    continue
                #write real bugs to db
                print("44444444444444444")
                new_analysis = CrashAnalysis(test_case_hash=f.split("/")[-1],crash_type=bug_type[0],frames_hash=stringtomd5(''.join([lines[j] for j in range(frame_start,frame_end+1)])))
                session.add(new_analysis)
                session.commit()
            except Exception as e:
                print(e)



analysis_crash_exec_result()
print("analysis crash end")

111111111111
22222222
analysis crash end
